In [5]:
import heapq
import copy
from copy import deepcopy
import time   # track elapsed time
import math   # for truncate elapsed time

In [6]:
class Node():
  def __init__(self, puzzle):
    self.puzzle = puzzle
    self.gn = 0
    self.hn = 0
    self.fn = 0
    self.goal = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

  # needed for heapq.heappush comparison https://stackoverflow.com/questions/7803121/in-python-heapq-heapify-doesnt-take-cmp-or-key-functions-as-arguments-like-sor
  def __lt__(self, other):
      return self.fn < other.fn

In [7]:
# globals
expandedNodes = 0
maxQueueSize = 0
visitedPuzzles = []
start = 0

In [8]:
def main():
    # reset globals
    global expandedNodes
    global maxQueueSize
    global visitedPuzzles
    global start
    expandedNodes = 0
    maxQueueSize = 0
    visitedPuzzles = []
    start = 0
    

    print("Welcome to my 8-Puzzle Solver.\n")
    puzzleChoice = int(input("Type '1' to use a default puzzle, or '2' to create your own.\n"))
    puzzle = []
    if (puzzleChoice == 1):
        puzzle = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]   # default puzzle is goal state
    elif (puzzleChoice == 2):
        print("Enter your puzzle, using a zero to represent the blank. Please only enter valid 8-puzzles. Enter the puzzle delimiting the numbers with a space. RET only when finished.\n")
        firstRow = input("Enter the first row: ").split()   # https://www.w3schools.com/python/ref_string_split.asp
        secondRow = input("Enter the second row: ").split()
        thirdRow = input("Enter the third row: ").split()

        # convert string to int https://www.kite.com/python/answers/how-to-convert-all-elements-of-a-list-to-int-in-python#:~:text=Use%20int()%20to%20convert,x)%20applied%20to%20all%20elements.
        firstRow = [int(value) for value in firstRow]
        secondRow = [int(value) for value in secondRow]
        thirdRow = [int(value) for value in thirdRow]

        puzzle = [firstRow, secondRow, thirdRow]
        
    algorithm = int(input("Select algorithm. (1) for Uniform Cost Search, (2) for the Misplaced Tile Heuristic, or (3) the Manhattan Distance Heuristic.\n"))
    
    puzzle = Node(puzzle)   # initialize puzzle

    start = time.clock()
    success = generalsearch(puzzle, algorithm)

    print()   # for new line separation
    if (success == 1):
      print("Solution found!")
    elif (success == 0):
      print("No solution found!")
    elif (success == 15):
      print("No solution found due to taking too long (more than 15 minutes)!")
    
    end = time.clock()
    print("Elapsed time: " + str(truncate(end - start, 4)) + " seconds")

In [9]:
def generalsearch(puzzle, algorithm):
  success = 0
  global maxQueueSize   # able to use and change global value
  nodes = []   # initial empty list
  heapq.heapify(nodes)   # https://www.geeksforgeeks.org/heap-queue-or-heapq-in-python/
  heapq.heappush(nodes, puzzle)   # push puzzle onto list
  initialExpansion = True

  while (True):
    if (len(nodes) == 0):
      success = 0
      break
    if (truncate((time.clock() - start), 4) > 900):   # fail if elapsed time greater than 15 minutes
      success = 15
      break
    node = heapq.heappop(nodes)
    if (node.puzzle == node.goal):
      print("\nGoal state!\n")
      print("Solution depth was " + str(node.gn))
      print("Number of nodes expanded: " + str(expandedNodes))
      print("Max queue size: " + str(maxQueueSize))
      success = 1
      break

    if (initialExpansion):
      print()   # for new line separation
      print("Initial state to expand...")
      outputPuzzle(node)
      initialExpansion = False
    
    print("Expanding...")
    nodes = expand(nodes, node, algorithm)

  return success

In [10]:
def expand(nodes, node, algorithm):
  moves = ["up", "left", "right", "down"]
  for curMove in moves:
    expandNode = copy.deepcopy(node) # deepcopy so changes are not made to original node https://www.programiz.com/python-programming/shallow-deep-copy
    i, j = getZeroIndex(expandNode)
    validMove = isValidMove(i, j, curMove)
    if (validMove):
      expandedNode = move(expandNode, i, j, curMove, node, algorithm)
      checkVisited(nodes, expandedNode)

  global expandedNodes   # able to change global value
  expandedNodes += 1

  global maxQueueSize   # able to change global value
  if (maxQueueSize < len(nodes)):
      maxQueueSize = len(nodes)

  cheapestSolution = nodes[0]
  print("The best state to expand with a g(n) = " + str(cheapestSolution.gn) + " and h(n) = " + str(cheapestSolution.hn) + " and f(n) = " + str(cheapestSolution.fn) + " is...")
  outputPuzzle(cheapestSolution)

  return nodes

In [11]:
# used code for truncating from https://www.delftstack.com/howto/python/python-truncate-float-python/
def truncate(number, decimal):
    integer = int(number * (10 ** decimal)) / (10 ** decimal)
    return float(integer)

def outputPuzzle(node):
  for i in node.puzzle:
    print(i)

def getZeroIndex(node):
  index = []
  for i, values in enumerate(node.puzzle):   # https://treyhunner.com/2016/04/how-to-loop-with-indexes-in-python/
    for j, value in enumerate(values):
      if (value == 0):
        index.append(i)
        index.append(j)

  return index

# values based on 8-puzzle dimensions
def isValidMove(i, j, move):
  isValid = True
  if (move == "up" and i == 0):
    isValid = False
  elif (move == "down" and i == 2):
    isValid = False
  elif (move == "left" and j == 0):
    isValid = False
  elif (move == "right" and j == 2):
    isValid = False

  return isValid

def move(expandNode, i, j, direction, node, algorithm):
  tempVal = expandNode.puzzle[i][j]
  if (direction == "up"):
    expandNode.puzzle[i][j] = expandNode.puzzle[i - 1][j]
    expandNode.puzzle[i - 1][j] = tempVal
  elif (direction == "left"):
    expandNode.puzzle[i][j] = expandNode.puzzle[i][j - 1]
    expandNode.puzzle[i][j - 1] = tempVal
  elif (direction == "right"):
    expandNode.puzzle[i][j] = expandNode.puzzle[i][j + 1]
    expandNode.puzzle[i][j + 1] = tempVal
  elif (direction == "down"):
    expandNode.puzzle[i][j] = expandNode.puzzle[i + 1][j]
    expandNode.puzzle[i + 1][j] = tempVal

  node = performAlgo(node, expandNode, algorithm)
  
  return expandNode

def performAlgo(node, expandNode, algorithm):
  if (algorithm == 1):
    hn = uniformCost()
  elif (algorithm == 2):
    hn = misplacedTile(expandNode)
  elif (algorithm == 3):
    hn = manhattanDistance(expandNode)
  expandNode.gn = node.gn + 1
  expandNode.hn = hn
  expandNode.fn = expandNode.gn + expandNode.hn

  return expandNode

def checkVisited(nodes, node):
  visited = False
  for i in visitedPuzzles:
    if (i == node):
      found = True
  if (not visited):
      heapq.heappush(nodes, node)
      visitedPuzzles.append(node)

In [12]:
def uniformCost():
  return 0

def misplacedTile(node):
  result = 0
  for i, values in enumerate(node.puzzle):   # https://treyhunner.com/2016/04/how-to-loop-with-indexes-in-python/
    for j, value in enumerate(values):
        if (value != node.goal[i][j]):
          if (value != 0):
            result += 1

  return result

def manhattanDistance(node):   # https://xlinux.nist.gov/dads/HTML/manhattanDistance.html#:~:text=Definition%3A%20The%20distance%20between%20two,y1%20%2D%20y2%7C.
  result = 0
  for i, values in enumerate(node.puzzle):
    for j, value in enumerate(values):
        k, l = getGoalIndex(node, value)
        manhattanFormula = (abs(k - i) + abs(l - j))
        result += manhattanFormula

  return result

def getGoalIndex(node, number):
  index = []
  for i, values in enumerate(node.goal):
    for j, value in enumerate(values):
      if (value == number):
        index.append(i)
        index.append(j)

  return index

In [18]:
main()

Welcome to my 8-Puzzle Solver.

Type '1' to use a default puzzle, or '2' to create your own.
2
Enter your puzzle, using a zero to represent the blank. Please only enter valid 8-puzzles. Enter the puzzle delimiting the numbers with a space. RET only when finished.

Enter the first row: 1 3 6
Enter the second row: 5 0 2
Enter the third row: 4 7 8
Select algorithm. (1) for Uniform Cost Search, (2) for the Misplaced Tile Heuristic, or (3) the Manhattan Distance Heuristic.
3

Initial state to expand...
[1, 3, 6]
[5, 0, 2]
[4, 7, 8]
Expanding...
The best state to expand with a g(n) = 1 and h(n) = 8 and f(n) = 9 is...
[1, 3, 6]
[5, 2, 0]
[4, 7, 8]
Expanding...
The best state to expand with a g(n) = 2 and h(n) = 8 and f(n) = 10 is...
[1, 3, 0]
[5, 2, 6]
[4, 7, 8]
Expanding...
The best state to expand with a g(n) = 2 and h(n) = 8 and f(n) = 10 is...
[1, 3, 6]
[5, 2, 8]
[4, 7, 0]
Expanding...
The best state to expand with a g(n) = 3 and h(n) = 8 and f(n) = 11 is...
[1, 0, 3]
[5, 2, 6]
[4, 7, 8]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
